In [10]:
from langchain.memory import ConversationBufferWindowMemory

#return_messages =True 는 쳇 모델, False는 쳇모델 아님 
memory = ConversationBufferWindowMemory(return_messages=True,k=4)

def add_message(input,output):
    memory.save_context({"input":input},{"output":output})
    
add_message("1","2")

In [11]:
add_message("3","4")

In [12]:
add_message("5","6")

In [13]:
add_message("7","8")

In [14]:
add_message("9","10")

In [15]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='4', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='5', additional_kwargs={}, response_metadata={}),
  AIMessage(content='6', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='7', additional_kwargs={}, response_metadata={}),
  AIMessage(content='8', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='9', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10', additional_kwargs={}, response_metadata={})]}